In [1]:
import pandas as pd
import geopandas
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
import random
from scipy import stats
from scipy import integrate
import glob
import seaborn as sns
# %matplotlib notebook
%matplotlib inline
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

from fitter import Fitter, get_common_distributions, get_distributions

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
def ACCR(i=0.05,n=20):
    return (i*((1+i)**n))/(((1+i)**n)-1)


def SpecPRecCost(flow, model, F, i=0.05, n=20, param=[]):
    # param = args
    if flow == 'LiquidManure' and model != 'Crystalactor':
        Cost = param[0] + param[1]*np.ceil(param[2]*F)*ACCR(i,n)*(1/F)
    elif flow == 'LiquidManure' and model == 'Crystalactor':
        Cost = param[0] + (param[1] + param[2]*np.ceil(param[3]*F))*np.ceil(param[4]*F)*ACCR(i,n)*(1/F)
    elif flow == 'SolidManure':
        Cost = param[0] + param[1]
    elif flow == 'PoultryLitter':
        Cost = param[0]
    elif flow == 'SlaughterhouseWasteLiquid':
        Cost = param[0] + param[1]*np.ceil(param[2]*F)*ACCR(i,n)*(1/F)
    elif flow == 'SlaughterhouseWasteSolid':
        Cost = param[0] + param[1]
    elif flow == 'WWTPLiquid':
        Cost = param[0]*((F/param[1])**param[2])*(1/F)
    elif flow == 'WWTPSludge':
        Cost = param[0]*((F/param[1])**param[2])*(1/F)
    elif flow == 'WWTPSSA':
        Cost = param[0] + param[1]
    elif flow == 'OFMSW':
        Cost = param[0] + param[1]
    else:
        raise ValueError('Flow no considered')
        
    return Cost
        


In [3]:
SpecPRecCost_Param = pd.read_csv('PRecvoeryCostPram.csv', sep=",", header=0, index_col=0)
SpecPRecCost_Param


,Multiform-LiquidManure,Crystalactor-LiquidManure,Ostara Pearl 500-LiquidManure,Ostara Pearl 2K-LiquidManure,Ostara Pearl 10K-LiquidManure,Nuresys-LiquidManure,MAPHEX-LiquidManure,Multiform-SlaughterhouseWasteLiquid,Ostara Pearl 500-SlaughterhouseWasteLiquid,Ostara Pearl 2K-SlaughterhouseWasteLiquid,Ostara Pearl 10K-SlaughterhouseWasteLiquid,Crystalactor-WWTPLiquid,Ostara Pearl-WWTPLiquid,P RoC-WWTPLiquid,REMNUT-WWTPLiquid,AirPrex-WWTPLiquid,PRISA-WWTPLiquid,Stuttgart process-WWTPSludge,Gifhorn process-WWTPSludge,PHOXNAN-WWTPSludge,Aqua Reci-WWTPSludge,MEPHREC-WWTPSludge,EcoPhos-SolidManure,AshDec depollution-SolidManure,AshDec Rhenania-SolidManure,PASCH-SolidManure,LEACHPHOS-SolidManure,RecoPhos-SolidManure,Thermophos-SolidManure,Quick wash-PoultryLitter,EcoPhos-SlaughterhouseWasteSolid,AshDec depollution-SlaughterhouseWasteSolid,AshDec Rhenania-SlaughterhouseWasteSolid,PASCH-SlaughterhouseWasteSolid,LEACHPHOS-SlaughterhouseWasteSolid,RecoPhos-SlaughterhouseWasteSolid,Thermophos-SlaughterhouseWasteSolid,EcoPhos-WWTPSSA,AshDec depollution-WWTPSSA,AshDec Rhenania-WWTPSSA,PASCH-WWTPSSA,LEACHPHOS-WWTPSSA,RecoPhos-WWTPSSA,Thermophos-WWTPSSA,Extraction-OFMSW,EcoPhos-OFMSW,AshDec depollution-OFMSW,AshDec Rhenania-OFMSW,PASCH-OFMSW,LEACHPHOS-OFMSW,RecoPhos-OFMSW,Thermophos-OFMSW
Param or flow,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
RecoveryEfficiency,1.800000e-01,1.800000e-01,1.800000e-01,1.800000e-01,1.800000e-01,1.800000e-01,0.180000,1.800000e-01,8.000000e-02,8.000000e-02,8.000000e-02,0.38,0.20,0.27,0.47,0.15,0.18,0.40,0.40,0.51,0.61,0.68,0.57,0.6,0.6,0.55,0.54,0.61,0.57,0.7,0.71,0.74,0.74,0.68,0.67,0.75,0.7,0.82,0.86,0.86,0.79,0.78,0.87,0.81,0.94,0.82,0.86,0.86,0.79,0.78,0.87,0.81
param1,2.570000e+01,3.530000e+00,1.257000e+01,1.257000e+01,1.257000e+01,1.037000e+01,184.670000,2.260000e+01,1.560000e+01,1.560000e+01,1.560000e+01,305920.00,130856.00,75970.00,977933.00,74195.00,186923.00,581730.00,400384.00,891667.00,939605.00,1154473.00,8.90,8.9,8.9,8.90,8.90,8.90,8.90,4.4,14.60,14.60,14.60,14.60,14.60,14.60,14.6,8.00,8.00,8.00,8.00,8.00,8.00,8.00,0.00,6.43,6.43,6.43,6.43,6.43,6.43,6.43
param2,1.100000e+06,2.300000e+06,2.300000e+06,3.100000e+06,1.000000e+07,1.380000e+06,300000.000000,1.100000e+06,2.300000e+06,3.100000e+06,1.000000e+07,24966.00,13140.00,17739.00,30879.00,9855.00,11826.00,26280.00,26280.00,33507.00,40077.00,44676.00,4.50,1.8,1.9,4.70,5.10,2.50,2.70,NaN,4.50,1.80,1.90,4.70,5.10,2.50,2.7,4.50,1.80,1.90,4.70,5.10,2.50,2.70,24.80,4.50,1.80,1.90,4.70,5.10,2.50,2.70
param3,1.190000e-04,7.100000e-01,7.020000e-05,1.830000e-05,3.650000e-06,2.240000e-05,0.000247,1.050000e-04,8.700000e-05,2.260000e-05,4.530000e-06,0.59,0.36,0.78,0.94,0.38,0.43,0.89,0.82,0.84,0.82,0.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
param4,NaN,3.320000e-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
param5,NaN,3.320000e-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Phosphorus recovered

# We are assuming SL separation is installed!!

### CattleCAFOs_Scenario1

In [18]:
CattleCAFOs_Scenario1 = pd.read_csv('CAFOSSizesDistributions/Scenario1/CAFOs_Scenario1_df.csv', sep=",", header=0, index_col=0)

ManureCattle = 33.0 #(kg/day/AU) Adjusted so that the results fit with the report, Fig 14
PCattle = 0.08/100

CattleCAFOs_Scenario1['kgPin/year'] = CattleCAFOs_Scenario1['Number of animals']*ManureCattle*365*PCattle

CattleCAFOs_Scenario1


,Number of animals,kgPin/year
0,1242.419120,11971.950644
1,4542.073913,43767.424229
2,725.013782,6986.232799
3,3461.014899,33350.339567
4,1012.318738,9754.703359
...,...,...
522,6052.622364,58323.069098
523,1880.211159,18117.714732
524,842.816232,8121.377207
525,2764.443698,26638.179473


In [23]:
PRecovery_CattleCAFOs_Scenario1 = CattleCAFOs_Scenario1.copy()

PFlow_Type = 'LiquidManure'

SpecPRecCost_Param_LiquidManure = SpecPRecCost_Param.filter(regex=PFlow_Type)

# for index, row in SpecPRecCost_Param_LiquidManure.iterrows():
for tech in SpecPRecCost_Param_LiquidManure.columns:
    name = tech.rsplit('-')[0]
    PRecovery_CattleCAFOs_Scenario1[name + ' kgPrec/year'] = (PRecovery_CattleCAFOs_Scenario1['kgPin/year']*
                                                              SpecPRecCost_Param_LiquidManure.loc['RecoveryEfficiency'][tech])
    
    TechParam = SpecPRecCost_Param.iloc[1:][tech].dropna().to_list()

    PRecovery_CattleCAFOs_Scenario1[name + ' EUR/kgPrec'] = SpecPRecCost(PFlow_Type, name, PRecovery_CattleCAFOs_Scenario1[name + ' kgPrec/year'],
                                                                         i=0.05, n=20, param=TechParam)
    
    PRecovery_CattleCAFOs_Scenario1[name + ' EUR/year'] = PRecovery_CattleCAFOs_Scenario1[name + ' EUR/kgPrec']*PRecovery_CattleCAFOs_Scenario1[name + ' kgPrec/year']
    
PFlow_Type = 'SolidManure'

SpecPRecCost_Param_SolidManure = SpecPRecCost_Param.filter(regex=PFlow_Type)

# for index, row in SpecPRecCost_Param_LiquidManure.iterrows():
for tech in SpecPRecCost_Param_SolidManure.columns:
    name = tech.rsplit('-')[0]
    PRecovery_CattleCAFOs_Scenario1[name + ' kgPrec/year'] = (PRecovery_CattleCAFOs_Scenario1['kgPin/year']*
                                                              SpecPRecCost_Param_SolidManure.loc['RecoveryEfficiency'][tech])
    
    TechParam = SpecPRecCost_Param.iloc[1:][tech].dropna().to_list()

    PRecovery_CattleCAFOs_Scenario1[name + ' EUR/kgPrec'] = SpecPRecCost(PFlow_Type, name, PRecovery_CattleCAFOs_Scenario1[name + ' kgPrec/year'],
                                                                         i=0.05, n=20, param=TechParam)
    
    PRecovery_CattleCAFOs_Scenario1[name + ' EUR/year'] = PRecovery_CattleCAFOs_Scenario1[name + ' EUR/kgPrec'] * PRecovery_CattleCAFOs_Scenario1[name + ' kgPrec/year']
    

PRecovery_CattleCAFOs_Scenario1.to_csv('Results/PRecoveryAndCosts/PRecovery_CattleCAFOs_Scenario1.csv')
PRecovery_CattleCAFOs_Scenario1

,Number of animals,kgPin/year,Multiform kgPrec/year,Multiform EUR/kgPrec,Multiform EUR/year,Crystalactor kgPrec/year,Crystalactor EUR/kgPrec,Crystalactor EUR/year,Ostara Pearl 500 kgPrec/year,Ostara Pearl 500 EUR/kgPrec,Ostara Pearl 500 EUR/year,Ostara Pearl 2K kgPrec/year,Ostara Pearl 2K EUR/kgPrec,Ostara Pearl 2K EUR/year,Ostara Pearl 10K kgPrec/year,Ostara Pearl 10K EUR/kgPrec,Ostara Pearl 10K EUR/year,Nuresys kgPrec/year,Nuresys EUR/kgPrec,Nuresys EUR/year,MAPHEX kgPrec/year,MAPHEX EUR/kgPrec,MAPHEX EUR/year,EcoPhos kgPrec/year,EcoPhos EUR/kgPrec,EcoPhos EUR/year,AshDec depollution kgPrec/year,AshDec depollution EUR/kgPrec,AshDec depollution EUR/year,AshDec Rhenania kgPrec/year,AshDec Rhenania EUR/kgPrec,AshDec Rhenania EUR/year,PASCH kgPrec/year,PASCH EUR/kgPrec,PASCH EUR/year,LEACHPHOS kgPrec/year,LEACHPHOS EUR/kgPrec,LEACHPHOS EUR/year,RecoPhos kgPrec/year,RecoPhos EUR/kgPrec,RecoPhos EUR/year,Thermophos kgPrec/year,Thermophos EUR/kgPrec,Thermophos EUR/year
0,1242.419120,11971.950644,2154.951116,66.660022,143649.089590,2154.951116,89.173710,192164.984950,2154.951116,98.213683,211645.686066,2154.951116,128.002790,275839.755819,2154.951116,384.933840,829513.607435,2154.951116,61.756210,133081.613396,2154.951116,195.840915,4.220276e+05,6824.011867,13.4,91441.759021,7183.170387,10.7,76859.923136,7183.170387,10.8,77578.240175,6584.572854,13.6,89550.190819,6464.853348,14.0,90507.946871,7302.889893,11.4,83252.944780,6824.011867,11.6,79158.537660
1,4542.073913,43767.424229,7878.136361,36.904026,290734.950393,7878.136361,26.956607,212367.828866,7878.136361,35.996600,283586.124599,7878.136361,44.144983,347780.194352,7878.136361,114.424783,901454.045968,7878.136361,24.425960,192431.044389,7878.136361,190.781287,1.503001e+06,24947.431811,13.4,334295.586262,26260.454537,10.7,280986.863551,26260.454537,10.8,283612.909004,24072.083326,13.6,327380.333233,23634.409084,14.0,330881.727172,26698.128780,11.4,304358.668089,24947.431811,11.6,289390.209002
2,725.013782,6986.232799,1257.521904,95.891100,120585.158836,1257.521904,150.293255,188997.059831,1257.521904,159.333209,200365.000869,1257.521904,210.381282,264559.070621,1257.521904,650.670911,818232.922237,1257.521904,98.427926,123775.272465,1257.521904,203.813027,2.562993e+05,3982.152695,13.4,53360.846116,4191.739679,10.7,44851.614567,4191.739679,10.8,45270.788535,3842.428039,13.6,52257.021334,3772.565711,14.0,52815.919958,4261.602007,11.4,48582.262882,3982.152695,11.6,46192.971265
3,3461.014899,33350.339567,6003.061122,40.403639,242545.516745,6003.061122,34.273983,205748.813272,6003.061122,43.313973,260016.428842,6003.061122,54.007529,324210.498595,6003.061122,146.239449,877884.350211,6003.061122,28.816384,172986.514158,6003.061122,192.690167,1.156731e+06,19009.693553,13.4,254729.893611,20010.203740,10.7,214109.180018,20010.203740,10.8,216110.200392,18342.686762,13.6,249460.539959,18009.183366,14.0,252128.567124,20343.707136,11.4,231918.261347,19009.693553,11.6,220512.445215
4,1012.318738,9754.703359,1755.846605,75.970249,133392.103650,1755.846605,108.640553,190756.146025,1755.846605,117.680521,206628.942360,1755.846605,154.240702,270823.012112,1755.846605,469.572263,824496.863728,1755.846605,73.436312,128942.899614,1755.846605,198.380068,3.483250e+05,5560.180915,13.4,74506.424259,5852.822016,10.7,62625.195568,5852.822016,10.8,63210.477769,5365.086848,13.6,72965.181129,5267.539814,14.0,73745.557397,5950.369049,11.4,67834.207161,5560.180915,11.6,64498.098613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522,6052.622364,58323.069098,10498.152438,42.515691,446336.209469,10498.152438,21.110046,221616.485616,10498.152438,30.150041,316519.726681,10498.152438,36.264838,380713.796433,10498.152438,89.004961,934387.648049,10498.152438,20.918025,219600.611102,10498.152438,191.549146,2.010912e+06,33244.149386,13.4,445471.601774,34993.841459,10.7,374434.103612,34993.8

### CattleCAFOs_Scenario2

In [21]:
CattleCAFOs_Scenario2 = pd.read_csv('CAFOSSizesDistributions/Scenario2/CAFOs_Scenario2_df.csv', sep=",", header=0, index_col=0)

ManureCattle = 33.0 #(kg/day/AU) Adjusted so that the results fit with the report, Fig 14
PCattle = 0.08/100

CattleCAFOs_Scenario2['kgPin/year'] = CattleCAFOs_Scenario2['Number of animals']*ManureCattle*365*PCattle

CattleCAFOs_Scenario2

,Number of animals,kgPin/year
0,1891.411476,18225.640980
1,4359.696014,42010.030788
2,1303.407175,12559.631541
3,1099.251849,10592.390814
4,3052.516945,29414.053284
...,...,...
693,1831.233422,17645.765251
694,359.846908,3467.484802
695,1454.301006,14013.644497
696,346.370518,3337.626313


In [24]:
PRecovery_CattleCAFOs_Scenario2 = CattleCAFOs_Scenario2.copy()

PFlow_Type = 'LiquidManure'

SpecPRecCost_Param_LiquidManure = SpecPRecCost_Param.filter(regex=PFlow_Type)

# for index, row in SpecPRecCost_Param_LiquidManure.iterrows():
for tech in SpecPRecCost_Param_LiquidManure.columns:
    name = tech.rsplit('-')[0]
    PRecovery_CattleCAFOs_Scenario2[name + ' kgPrec/year'] = (PRecovery_CattleCAFOs_Scenario2['kgPin/year']*
                                                              SpecPRecCost_Param_LiquidManure.loc['RecoveryEfficiency'][tech])
    
    TechParam = SpecPRecCost_Param.iloc[1:][tech].dropna().to_list()

    PRecovery_CattleCAFOs_Scenario2[name + ' EUR/kgPrec'] = SpecPRecCost(PFlow_Type, name, PRecovery_CattleCAFOs_Scenario2[name + ' kgPrec/year'],
                                                                         i=0.05, n=20, param=TechParam)
    
    PRecovery_CattleCAFOs_Scenario2[name + ' EUR/year'] = PRecovery_CattleCAFOs_Scenario2[name + ' EUR/kgPrec']* PRecovery_CattleCAFOs_Scenario2[name + ' kgPrec/year']
    
PFlow_Type = 'SolidManure'

SpecPRecCost_Param_SolidManure = SpecPRecCost_Param.filter(regex=PFlow_Type)

# for index, row in SpecPRecCost_Param_LiquidManure.iterrows():
for tech in SpecPRecCost_Param_SolidManure.columns:
    name = tech.rsplit('-')[0]
    PRecovery_CattleCAFOs_Scenario2[name + ' kgPrec/year'] = (PRecovery_CattleCAFOs_Scenario2['kgPin/year']*
                                                              SpecPRecCost_Param_SolidManure.loc['RecoveryEfficiency'][tech])
    
    TechParam = SpecPRecCost_Param.iloc[1:][tech].dropna().to_list()

    PRecovery_CattleCAFOs_Scenario2[name + ' EUR/kgPrec'] = SpecPRecCost(PFlow_Type, name, PRecovery_CattleCAFOs_Scenario2[name + ' kgPrec/year'],
                                                                         i=0.05, n=20, param=TechParam)
    
    PRecovery_CattleCAFOs_Scenario2[name + ' EUR/year'] = PRecovery_CattleCAFOs_Scenario2[name + ' EUR/kgPrec']*PRecovery_CattleCAFOs_Scenario2[name + ' kgPrec/year']
    
PRecovery_CattleCAFOs_Scenario2.to_csv('Results/PRecoveryAndCosts/PRecovery_CattleCAFOs_Scenario2.csv')

PRecovery_CattleCAFOs_Scenario2

,Number of animals,kgPin/year,Multiform kgPrec/year,Multiform EUR/kgPrec,Multiform EUR/year,Crystalactor kgPrec/year,Crystalactor EUR/kgPrec,Crystalactor EUR/year,Ostara Pearl 500 kgPrec/year,Ostara Pearl 500 EUR/kgPrec,Ostara Pearl 500 EUR/year,Ostara Pearl 2K kgPrec/year,Ostara Pearl 2K EUR/kgPrec,Ostara Pearl 2K EUR/year,Ostara Pearl 10K kgPrec/year,Ostara Pearl 10K EUR/kgPrec,Ostara Pearl 10K EUR/year,Nuresys kgPrec/year,Nuresys EUR/kgPrec,Nuresys EUR/year,MAPHEX kgPrec/year,MAPHEX EUR/kgPrec,MAPHEX EUR/year,EcoPhos kgPrec/year,EcoPhos EUR/kgPrec,EcoPhos EUR/year,AshDec depollution kgPrec/year,AshDec depollution EUR/kgPrec,AshDec depollution EUR/year,AshDec Rhenania kgPrec/year,AshDec Rhenania EUR/kgPrec,AshDec Rhenania EUR/year,PASCH kgPrec/year,PASCH EUR/kgPrec,PASCH EUR/year,LEACHPHOS kgPrec/year,LEACHPHOS EUR/kgPrec,LEACHPHOS EUR/year,RecoPhos kgPrec/year,RecoPhos EUR/kgPrec,RecoPhos EUR/year,Thermophos kgPrec/year,Thermophos EUR/kgPrec,Thermophos EUR/year
0,1891.411476,18225.640980,3280.615376,52.605576,172578.661083,3280.615376,59.787131,196138.579790,3280.615376,68.827113,225795.285820,3280.615376,88.394805,289989.355573,3280.615376,257.166144,843663.207188,3280.615376,44.124268,144754.751776,3280.615376,192.007884,6.299040e+05,10388.615359,13.4,139207.445805,10935.384588,10.7,117008.615092,10935.384588,10.8,118102.153551,10024.102539,13.6,136327.794531,9841.846129,14.0,137785.845809,11117.640998,11.4,126741.107375,10388.615359,11.6,120507.938160
1,4359.696014,42010.030788,7561.805542,37.372721,282605.248337,7561.805542,27.936606,211251.181074,7561.805542,36.976598,279609.846201,7561.805542,45.465850,343803.915953,7561.805542,118.685645,897477.767569,7561.805542,25.013959,189150.693793,7561.805542,191.036939,1.444584e+06,23945.717549,13.4,320872.615162,25206.018473,10.7,269704.397662,25206.018473,10.8,272224.999509,23105.516934,13.6,314235.030298,22685.416626,14.0,317595.832761,25626.118781,11.4,292137.754103,23945.717549,11.6,277770.323573
2,1303.407175,12559.631541,2260.733677,64.743452,146367.701417,2260.733677,85.166333,192538.397392,2260.733677,94.206308,212975.372863,2260.733677,122.601545,277169.442615,2260.733677,367.510469,830843.294231,2260.733677,59.351785,134178.578557,2260.733677,195.318214,4.415625e+05,7158.989978,13.4,95930.465708,7535.778924,10.7,80632.834491,7535.778924,10.8,81386.412384,6907.797347,13.6,93946.043924,6782.201032,14.0,94950.814448,7661.375240,11.4,87339.677734,7158.989978,11.6,83044.283747
3,1099.251849,10592.390814,1906.630347,71.994682,137267.245815,1906.630347,100.328002,191288.412634,1906.630347,109.367972,208524.293994,1906.630347,143.036832,272718.363747,1906.630347,433.430747,826392.215362,1906.630347,68.448783,130506.527016,1906.630347,197.295822,3.761702e+05,6037.662764,13.4,80904.681037,6355.434488,10.7,68003.149025,6355.434488,10.8,68638.692474,5825.814948,13.6,79231.083288,5719.891040,14.0,80078.474553,6461.358396,11.4,73659.485720,6037.662764,11.6,70036.888062
4,3052.516945,29414.053284,5294.529591,42.371329,224336.256401,5294.529591,38.388245,203247.696967,5294.529591,47.428234,251110.187499,5294.529591,59.552837,315304.257251,5294.529591,164.127538,868978.108867,5294.529591,31.284940,165639.042183,5294.529591,193.763452,1.025886e+06,16766.010372,13.4,224664.538983,17648.431970,10.7,188838.222083,17648.431970,10.8,190603.065280,16177.729306,13.6,220017.118564,15883.588773,14.0,222370.242827,17942.572503,11.4,204545.326537,16766.010372,11.6,194485.720314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,1831.233422,17645.765251,3176.237745,53.489748,169896.155961,3176.237745,61.635854,195770.126751,3176.237745,70.675836,224483.258996,3176.237745,90.886562,288677.328748,3176.237745,265.204071,842351.180364,3176.237745,45.233502,143672.355741,3176.237745,192.249022,6.106286e+05,10058.086193,13.4,134778.354988,10587.459151,10.7,113285.812912,

In [52]:
### SwineCAFOs_Scenario2

In [26]:
SwineCAFOs_Scenario1 = pd.read_csv('CAFOSSizesDistributions/Swine/CAFOs_Scenario1_df.csv', sep=",", header=0, index_col=0)

ManureSwine = 33.1 #(kg/day/AU) Adjusted so that the results fit with the report, Fig 14
PSwine = 0.14/100

SwineCAFOs_Scenario1['kgPin/year'] = SwineCAFOs_Scenario1['Number of animals']*ManureSwine*365*PSwine

SwineCAFOs_Scenario1

,Number of animals,kgPin/year
0,463.685456,7842.822176
1,1030.150829,17424.074137
2,633.662642,10717.833285
3,1052.466189,17801.518370
4,681.793512,11531.923636
...,...,...
581,629.966461,10655.315713
582,499.007334,8440.259954
583,680.540557,11510.731040
584,1199.778944,20293.181040


In [39]:
PRecovery_SwineCAFOs_Scenario1 = SwineCAFOs_Scenario1.copy()

PFlow_Type = 'LiquidManure'

SpecPRecCost_Param_LiquidManure = SpecPRecCost_Param.filter(regex=PFlow_Type)

# for index, row in SpecPRecCost_Param_LiquidManure.iterrows():
for tech in SpecPRecCost_Param_LiquidManure.columns:
    name = tech.rsplit('-')[0]
    PRecovery_SwineCAFOs_Scenario1[name + ' kgPrec/year'] = (PRecovery_SwineCAFOs_Scenario1['kgPin/year']*
                                                              SpecPRecCost_Param_LiquidManure.loc['RecoveryEfficiency'][tech])
    
    TechParam = SpecPRecCost_Param.iloc[1:][tech].dropna().to_list()

    PRecovery_SwineCAFOs_Scenario1[name + ' EUR/kgPrec'] = SpecPRecCost(PFlow_Type, name, PRecovery_SwineCAFOs_Scenario1[name + ' kgPrec/year'],
                                                                         i=0.05, n=20, param=TechParam)
    
    PRecovery_SwineCAFOs_Scenario1[name + ' EUR/year'] = PRecovery_SwineCAFOs_Scenario1[name + ' EUR/kgPrec']*PRecovery_SwineCAFOs_Scenario1[name + ' kgPrec/year']
    
PFlow_Type = 'SolidManure'

SpecPRecCost_Param_SolidManure = SpecPRecCost_Param.filter(regex=PFlow_Type)

# for index, row in SpecPRecCost_Param_LiquidManure.iterrows():
for tech in SpecPRecCost_Param_SolidManure.columns:
    name = tech.rsplit('-')[0]
    PRecovery_SwineCAFOs_Scenario1[name + ' kgPrec/year'] = (PRecovery_SwineCAFOs_Scenario1['kgPin/year']*
                                                              SpecPRecCost_Param_SolidManure.loc['RecoveryEfficiency'][tech])
    
    TechParam = SpecPRecCost_Param.iloc[1:][tech].dropna().to_list()

    PRecovery_SwineCAFOs_Scenario1[name + ' EUR/kgPrec'] = SpecPRecCost(PFlow_Type, name, PRecovery_SwineCAFOs_Scenario1[name + ' kgPrec/year'],
                                                                         i=0.05, n=20, param=TechParam)
    
    PRecovery_SwineCAFOs_Scenario1[name + ' EUR/year'] = PRecovery_SwineCAFOs_Scenario1[name + ' EUR/kgPrec']*PRecovery_SwineCAFOs_Scenario1[name + ' kgPrec/year']
    

PRecovery_SwineCAFOs_Scenario1.to_csv('Results/PRecoveryAndCosts/PRecovery_SwineCAFOs_Scenario1.csv')

PRecovery_SwineCAFOs_Scenario1

,Number of animals,kgPin/year,Multiform kgPrec/year,Multiform EUR/kgPrec,Multiform EUR/year,Crystalactor kgPrec/year,Crystalactor EUR/kgPrec,Crystalactor EUR/year,Ostara Pearl 500 kgPrec/year,Ostara Pearl 500 EUR/kgPrec,Ostara Pearl 500 EUR/year,Ostara Pearl 2K kgPrec/year,Ostara Pearl 2K EUR/kgPrec,Ostara Pearl 2K EUR/year,Ostara Pearl 10K kgPrec/year,Ostara Pearl 10K EUR/kgPrec,Ostara Pearl 10K EUR/year,Nuresys kgPrec/year,Nuresys EUR/kgPrec,Nuresys EUR/year,MAPHEX kgPrec/year,MAPHEX EUR/kgPrec,MAPHEX EUR/year,EcoPhos kgPrec/year,EcoPhos EUR/kgPrec,EcoPhos EUR/year,AshDec depollution kgPrec/year,AshDec depollution EUR/kgPrec,AshDec depollution EUR/year,AshDec Rhenania kgPrec/year,AshDec Rhenania EUR/kgPrec,AshDec Rhenania EUR/year,PASCH kgPrec/year,PASCH EUR/kgPrec,PASCH EUR/year,LEACHPHOS kgPrec/year,LEACHPHOS EUR/kgPrec,LEACHPHOS EUR/year,RecoPhos kgPrec/year,RecoPhos EUR/kgPrec,RecoPhos EUR/year,Thermophos kgPrec/year,Thermophos EUR/kgPrec,Thermophos EUR/year
0,463.685456,7842.822176,1411.707992,88.224861,124547.741294,1411.707992,134.263841,189541.336721,1411.707992,143.303800,202303.119993,1411.707992,188.776426,266497.189745,1411.707992,580.977827,820171.041361,1411.707992,88.810280,125374.182196,1411.707992,201.722235,284772.890965,4470.408640,13.4,59903.475777,4705.693305,10.7,50350.918367,4705.693305,10.8,50821.487697,4313.552197,13.6,58664.309873,4235.123975,14.0,59291.735647,4784.121527,11.4,54538.985409,4470.408640,11.6,51856.740225
1,1030.150829,17424.074137,3136.333345,53.843324,168870.612868,3136.333345,62.375150,195629.264218,3136.333345,71.415132,223981.660681,3136.333345,91.883004,288175.730434,3136.333345,268.418401,841849.582049,3136.333345,45.677079,143258.547107,3136.333345,192.345452,603259.454917,9931.722258,13.4,133085.078259,10454.444482,10.7,111862.555960,10454.444482,10.8,112908.000408,9583.240775,13.6,130332.074545,9409.000034,14.0,131726.000476,10628.685224,11.4,121167.011549,9931.722258,11.6,115207.978194
2,633.662642,10717.833285,1929.209991,71.452845,137847.542687,1929.209991,99.195069,191368.118780,1929.209991,108.235040,208808.120130,1929.209991,141.509836,273002.189882,1929.209991,428.504956,826676.041498,1929.209991,67.769024,130740.677933,1929.209991,197.148049,380339.985259,6109.164973,13.4,81862.810633,6430.699971,10.7,68808.489691,6430.699971,10.8,69451.559688,5894.808307,13.6,80169.392974,5787.629974,14.0,81026.819636,6537.878304,11.4,74531.812666,6109.164973,11.6,70866.313682
3,1052.466189,17801.518370,3204.273307,53.246603,170616.669890,3204.273307,61.127461,195869.092283,3204.273307,70.167443,224835.666003,3204.273307,90.201337,289029.735756,3204.273307,262.993667,842703.587371,3204.273307,44.928466,143963.084513,3204.273307,192.182710,615805.927694,10146.865471,13.4,135967.997312,10680.911022,10.7,114285.747937,10680.911022,10.8,115353.839039,9790.835104,13.6,133155.357409,9612.819920,14.0,134579.478879,10858.926206,11.4,123791.758747,10146.865471,11.6,117703.639464
4,681.793512,11531.923636,2075.746254,68.222946,141613.524648,2075.746254,92.441642,191885.391789,2075.746254,101.481614,210650.080956,2075.746254,132.407393,274844.150709,2075.746254,399.142236,828518.002325,2075.746254,63.716969,132260.258981,2075.746254,196.267167,407400.836955,6573.196472,13.4,88080.832728,6919.154181,10.7,74034.949740,6919.154181,10.8,74726.865158,6342.558000,13.6,86258.788794,6227.238763,14.0,87181.342684,7034.473418,11.4,80192.996961,6573.196472,11.6,76249.079078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,629.966461,10655.315713,1917.956828,71.721289,137558.336396,1917.956828,99.756362,191328.395115,1917.956828,108.796332,208666.667870,1917.956828,142.266361,272860.737623,1917.956828,430.945357,826534.589238,1917.956828,68.105799,130623.982632,1917.956828,197.221261,378261.863634,6073.529956,13.4,81385.301413,6393.189428,10.7,68407.126875,6393.189428,10.8,

### WWTPs Liquid This database just reports releases, not disposals, so we will create a new dataframe in cells below for it

In [28]:
#This database just reports releases, not disposals, so we will create a new dataframe in cells below for it
WWWTPsLiquid_Ontario_df = pd.read_csv('WWTPsData/wser_effluent (1).csv', sep=",", header=0, index_col=0) 

PWWTP_Liquid = 0.000452629303341566*1000 #kg/thousan m3; source: mwd.csv, the other table of Jorge

WWWTPsLiquid_Ontario_df['kgPin/year'] = WWWTPsLiquid_Ontario_df['Y2019']*PWWTP_Liquid

WWWTPsLiquid_Ontario_df

,Owner Name,System Name,City,Province,Latitude,Longitude,System status,CDUID,CDNAME,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Units,kgPin/year
ID_WSER,,,,,,,,,,,,,,,,,,
375,Stelco Inc.,Lake Erie,Haldimand County,ON,42.7940,-80.1090,operational,3528.0,Haldimand-Norfolk,162.3130,164.7630,135.9750,125.5630,161.7000,202.1250,187.6450,thousand m3,84.933626
503,Regional Municipality of Durham,Duffin Creek WPCP,Pickering,ON,43.8156,-79.0484,operational,3518.0,Durham,125615.3020,125187.3930,116781.0590,116277.1300,121734.3420,125252.9370,124847.1070,thousand m3,56509.459066
1412,Bruce Power LP,Bruce Power,Tiverton,ON,44.3233,-81.5875,operational,3541.0,Bruce,376.6781,411.1270,411.6526,389.8714,391.9040,353.0745,340.7550,thousand m3,154.235698
1436,The Corporation of the City of Barrie,Water Pollution Control Centre,Barrie,ON,44.3776,-79.6920,operational,3543.0,Simcoe,18676.6950,18465.9400,17527.8700,17707.7790,18509.8430,18110.8980,18197.0700,thousand m3,8236.527117
7059,City of Sarnia,Water Pollution Control Centre,Sarnia,ON,42.9528,-82.4065,operational,3538.0,Lambton,8504.6068,7976.2739,8365.6310,9427.2610,10084.1690,10682.9760,11979.8060,thousand m3,5422.411244
7391,DEPARTMENT OF NATIONAL DEFENCE,4th Canadian Division Training Centre,Meaford,ON,44.6653,-80.6754,operational,3542.0,Grey,103.0550,96.5260,93.4260,113.8270,113.8650,112.5050,106.9140,thousand m3,48.392409
7890,The Corporation of the City of Belleville,Belleville Wastewater Treatment Facility,Belleville,ON,44.1572,-77.3725,operational,3512.0,Hastings,9264.7210,10250.0387,8143.5230,7614.1800,12558.3369,11844.7320,12071.0880,thousand m3,5463.728152
8469,Canadian Nuclear Laboratories,Chalk River Laboratories,Chalk River,ON,46.0524,-77.3618,operational,3547.0,Renfrew,207.0580,208.8070,210.0190,205.7020,227.4760,210.6520,166.0380,thousand m3,75.153664
8493,The Corporation of the Town of Kirkland Lake,Kirkland Lake Wastewater Treatment Plant,Kirkland Lake,ON,48.1372,-80.0363,operational,3554.0,Timiskaming,2981.9380,3228.6470,3246.8350,2629.0720,3309.3116,3332.0830,4116.5490,thousand m3,1863.270706


In [29]:
PRecovery_WWWTPsLiquid_Ontario_df = WWWTPsLiquid_Ontario_df.copy()

PFlow_Type = 'WWTPLiquid'

SpecPRecCost_Param_WWTPLiquid = SpecPRecCost_Param.filter(regex=PFlow_Type)

# for index, row in SpecPRecCost_Param_LiquidManure.iterrows():
for tech in SpecPRecCost_Param_WWTPLiquid.columns:
    name = tech.rsplit('-')[0]
    PRecovery_WWWTPsLiquid_Ontario_df[name + ' kgPrec/year'] = (PRecovery_WWWTPsLiquid_Ontario_df['kgPin/year']*
                                                              SpecPRecCost_Param_WWTPLiquid.loc['RecoveryEfficiency'][tech])
    
    TechParam = SpecPRecCost_Param.iloc[1:][tech].dropna().to_list()

    PRecovery_WWWTPsLiquid_Ontario_df[name + ' EUR/kgPrec'] = SpecPRecCost(PFlow_Type, name, PRecovery_WWWTPsLiquid_Ontario_df[name + ' kgPrec/year'],
                                                                         i=0.05, n=20, param=TechParam)
    
    PRecovery_WWWTPsLiquid_Ontario_df[name + ' EUR/year'] = PRecovery_WWWTPsLiquid_Ontario_df[name + ' EUR/kgPrec']*PRecovery_WWWTPsLiquid_Ontario_df[name + ' kgPrec/year']

PRecovery_WWWTPsLiquid_Ontario_df.to_csv('Results/PRecoveryAndCosts/PRecovery_WWWTPsLiquid_Ontario_df.csv')

PRecovery_WWWTPsLiquid_Ontario_df

,Owner Name,System Name,City,Province,Latitude,Longitude,System status,CDUID,CDNAME,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Units,kgPin/year,Crystalactor kgPrec/year,Crystalactor EUR/kgPrec,Crystalactor EUR/year,Ostara Pearl kgPrec/year,Ostara Pearl EUR/kgPrec,Ostara Pearl EUR/year,P RoC kgPrec/year,P RoC EUR/kgPrec,P RoC EUR/year,REMNUT kgPrec/year,REMNUT EUR/kgPrec,REMNUT EUR/year,AirPrex kgPrec/year,AirPrex EUR/kgPrec,AirPrex EUR/year,PRISA kgPrec/year,PRISA EUR/kgPrec,PRISA EUR/year
ID_WSER,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
375,Stelco Inc.,Lake Erie,Haldimand County,ON,42.7940,-80.1090,operational,3528.0,Haldimand-Norfolk,162.3130,164.7630,135.9750,125.5630,161.7000,202.1250,187.6450,thousand m3,84.933626,32.274778,187.300115,6045.069580,16.986725,702.795751,11938.198246,22.932079,18.500346,424.251392,39.918804,47.201415,1.884224e+03,12.740044,465.135137,5925.842043,15.288053,700.263871,10705.670906
503,Regional Municipality of Durham,Duffin Creek WPCP,Pickering,ON,43.8156,-79.0484,operational,3518.0,Durham,125615.3020,125187.3930,116781.0590,116277.1300,121734.3420,125252.9370,124847.1070,thousand m3,56509.459066,21473.594445,13.034400,279895.412932,11301.891813,10.966867,123946.344166,15257.553948,4.427012,67545.377805,26559.445761,31.957480,8.487730e+05,8476.418860,8.265962,70065.759776,10171.702632,17.223764,175195.008381
1412,Bruce Power LP,Bruce Power,Tiverton,ON,44.3233,-81.5875,operational,3541.0,Bruce,376.6781,411.1270,411.6526,389.8714,391.9040,353.0745,340.7550,thousand m3,154.235698,58.609565,146.657709,8595.544556,30.847140,479.735408,14798.465115,41.643639,16.224701,675.655573,72.490778,45.541644,3.301349e+03,23.135355,321.317145,7433.786129,27.762426,498.392770,13836.592232
1436,The Corporation of the City of Barrie,Water Pollution Control Centre,Barrie,ON,44.3776,-79.6920,operational,3543.0,Simcoe,18676.6950,18465.9400,17527.8700,17707.7790,18509.8430,18110.8980,18197.0700,thousand m3,8236.527117,3129.880304,28.708177,89853.157018,1647.305423,37.615243,61963.793131,2223.862322,6.762599,15039.090112,3871.167745,35.871963,1.388664e+05,1235.479068,27.280177,33704.087152,1482.574881,51.625346,76538.440783
7059,City of Sarnia,Water Pollution Control Centre,Sarnia,ON,42.9528,-82.4065,operational,3538.0,Lambton,8504.6068,7976.2739,8365.6310,9427.2610,10084.1690,10682.9760,11979.8060,thousand m3,5422.411244,2060.516273,34.075466,70213.051683,1084.482249,49.153782,53306.404485,1464.051036,7.414042,10854.535685,2548.533285,36.783093,9.374294e+04,813.361687,35.351612,28753.646588,976.034024,65.516009,63945.853451
7391,DEPARTMENT OF NATIONAL DEFENCE,4th Canadian Division Training Centre,Meaford,ON,44.6653,-80.6754,operational,3542.0,Grey,103.0550,96.5260,93.4260,113.8270,113.8650,112.5050,106.9140,thousand m3,48.392409,18.389116,235.885929,4337.733607,9.678482,1007.355299,9749.669991,13.065951,20.937575,273.569319,22.744432,48.821730,1.110423e+03,7.258861,659.244701,4785.365915,8.710634,964.970638,8405.505740
7890,The Corporation of the City of Belleville,Belleville Wastewater Treatment Facility,Belleville,ON,44.1572,-77.3725,operational,3512.0,Hastings,9264.7210,10250.0387,8143.5230,7614.1800,12558.3369,11844.7320,12071.0880,thousand m3,5463.728152,2076.216698,33.969580,70528.209971,1092.745630,48.915568,53452.272954,1475.206601,7.401671,10918.993885,2567.952231,36.766344,9.441422e+04,819.559223,35.185628,28836.706036,983.471067,65.233150,64154.915796
8469,Canadian Nuclear Laboratories,Chalk River Laboratories,Chalk River,ON,46.0524,-77.3618,operational,3547.0,Renfrew,207.0580,208.8070,210.0190,205.7020,227.4760,210.6520,166.0380,thousand m3,75.153664,28.558392,196.934199,5624.124126,15.030733,760.032230,11423.841408,20.291489,19.005020,385.640168,35.322222,47.549153,1.679542e+03,11.273050,501.787040,5656.670210,13.527660,750.836778,10157.064319
8493,The Corporation of the Town of Kirkland Lake,Kirkland Lake Wastewater Treatment Plant,Kirkland Lake,ON,48.1372,-80.0363,operational,3554.0,Timiskaming,2981.9380,3228.

### WWTPs Sludge 

In [31]:
WWWTPsSludge_Ontario_df = pd.read_csv('WWTPsData/wser_effluent (1).csv', sep=",", header=0, index_col=0) 

PWWTP_Liquid = 0.000452629303341566*1000 #kg/thousan m3; source: mwd.csv, the other table of Jorge
P_LiquidFraction = 0.141
P_SludgeFraction = 0.859

WWWTPsSludge_Ontario_df['kgPin/year'] = (WWWTPsSludge_Ontario_df['Y2019']*PWWTP_Liquid*P_SludgeFraction)/P_LiquidFraction

WWWTPsSludge_Ontario_df

,Owner Name,System Name,City,Province,Latitude,Longitude,System status,CDUID,CDNAME,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Units,kgPin/year
ID_WSER,,,,,,,,,,,,,,,,,,
375,Stelco Inc.,Lake Erie,Haldimand County,ON,42.7940,-80.1090,operational,3528.0,Haldimand-Norfolk,162.3130,164.7630,135.9750,125.5630,161.7000,202.1250,187.6450,thousand m3,517.432514
503,Regional Municipality of Durham,Duffin Creek WPCP,Pickering,ON,43.8156,-79.0484,operational,3518.0,Durham,125615.3020,125187.3930,116781.0590,116277.1300,121734.3420,125252.9370,124847.1070,thousand m3,344266.846364
1412,Bruce Power LP,Bruce Power,Tiverton,ON,44.3233,-81.5875,operational,3541.0,Bruce,376.6781,411.1270,411.6526,389.8714,391.9040,353.0745,340.7550,thousand m3,939.634502
1436,The Corporation of the City of Barrie,Water Pollution Control Centre,Barrie,ON,44.3776,-79.6920,operational,3543.0,Simcoe,18676.6950,18465.9400,17527.8700,17707.7790,18509.8430,18110.8980,18197.0700,thousand m3,50178.558819
7059,City of Sarnia,Water Pollution Control Centre,Sarnia,ON,42.9528,-82.4065,operational,3538.0,Lambton,8504.6068,7976.2739,8365.6310,9427.2610,10084.1690,10682.9760,11979.8060,thousand m3,33034.406089
7391,DEPARTMENT OF NATIONAL DEFENCE,4th Canadian Division Training Centre,Meaford,ON,44.6653,-80.6754,operational,3542.0,Grey,103.0550,96.5260,93.4260,113.8270,113.8650,112.5050,106.9140,thousand m3,294.816168
7890,The Corporation of the City of Belleville,Belleville Wastewater Treatment Facility,Belleville,ON,44.1572,-77.3725,operational,3512.0,Hastings,9264.7210,10250.0387,8143.5230,7614.1800,12558.3369,11844.7320,12071.0880,thousand m3,33286.116898
8469,Canadian Nuclear Laboratories,Chalk River Laboratories,Chalk River,ON,46.0524,-77.3618,operational,3547.0,Renfrew,207.0580,208.8070,210.0190,205.7020,227.4760,210.6520,166.0380,thousand m3,457.851047
8493,The Corporation of the Town of Kirkland Lake,Kirkland Lake Wastewater Treatment Plant,Kirkland Lake,ON,48.1372,-80.0363,operational,3554.0,Timiskaming,2981.9380,3228.6470,3246.8350,2629.0720,3309.3116,3332.0830,4116.5490,thousand m3,11351.415152


In [32]:
PRecovery_WWWTPsSludge_Ontario_df = WWWTPsSludge_Ontario_df.copy()

PFlow_Type = 'WWTPSludge' 

SpecPRecCost_Param_WWTPSludge = SpecPRecCost_Param.filter(regex=PFlow_Type)

# for index, row in SpecPRecCost_Param_LiquidManure.iterrows():
for tech in SpecPRecCost_Param_WWTPSludge.columns:
    name = tech.rsplit('-')[0]
    PRecovery_WWWTPsSludge_Ontario_df[name + ' kgPrec/year'] = (PRecovery_WWWTPsSludge_Ontario_df['kgPin/year']*
                                                              SpecPRecCost_Param_WWTPSludge.loc['RecoveryEfficiency'][tech])
    
    TechParam = SpecPRecCost_Param.iloc[1:][tech].dropna().to_list()

    PRecovery_WWWTPsSludge_Ontario_df[name + ' EUR/kgPrec'] = SpecPRecCost(PFlow_Type, name, PRecovery_WWWTPsSludge_Ontario_df[name + ' kgPrec/year'],
                                                                         i=0.05, n=20, param=TechParam)
    
    PRecovery_WWWTPsSludge_Ontario_df[name + ' EUR/year'] =  PRecovery_WWWTPsSludge_Ontario_df[name + ' EUR/kgPrec']*PRecovery_WWWTPsSludge_Ontario_df[name + ' kgPrec/year']
    
PFlow_Type = 'WWTPSSA'

SpecPRecCost_Param_WWTPSSA = SpecPRecCost_Param.filter(regex=PFlow_Type)

# for index, row in SpecPRecCost_Param_LiquidManure.iterrows():
for tech in SpecPRecCost_Param_WWTPSSA.columns:
    name = tech.rsplit('-')[0]
    PRecovery_WWWTPsSludge_Ontario_df[name + ' kgPrec/year'] = (PRecovery_WWWTPsSludge_Ontario_df['kgPin/year']*
                                                              SpecPRecCost_Param_WWTPSSA.loc['RecoveryEfficiency'][tech])
    
    TechParam = SpecPRecCost_Param.iloc[1:][tech].dropna().to_list()

    PRecovery_WWWTPsSludge_Ontario_df[name + ' EUR/kgPrec'] = SpecPRecCost(PFlow_Type, name, PRecovery_WWWTPsSludge_Ontario_df[name + ' kgPrec/year'],
                                                                         i=0.05, n=20, param=TechParam)
    
    PRecovery_WWWTPsSludge_Ontario_df[name + ' EUR/year'] = PRecovery_WWWTPsSludge_Ontario_df[name + ' EUR/kgPrec']*PRecovery_WWWTPsSludge_Ontario_df[name + ' kgPrec/year']
    

PRecovery_WWWTPsSludge_Ontario_df.to_csv('Results/PRecoveryAndCosts/PRecovery_WWWTPsSludge_Ontario_df.csv')

PRecovery_WWWTPsSludge_Ontario_df

,Owner Name,System Name,City,Province,Latitude,Longitude,System status,CDUID,CDNAME,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Units,kgPin/year,Stuttgart process kgPrec/year,Stuttgart process EUR/kgPrec,Stuttgart process EUR/year,Gifhorn process kgPrec/year,Gifhorn process EUR/kgPrec,Gifhorn process EUR/year,PHOXNAN kgPrec/year,PHOXNAN EUR/kgPrec,PHOXNAN EUR/year,Aqua Reci kgPrec/year,Aqua Reci EUR/kgPrec,Aqua Reci EUR/year,MEPHREC kgPrec/year,MEPHREC EUR/kgPrec,MEPHREC EUR/year,EcoPhos kgPrec/year,EcoPhos EUR/kgPrec,EcoPhos EUR/year,AshDec depollution kgPrec/year,AshDec depollution EUR/kgPrec,AshDec depollution EUR/year,AshDec Rhenania kgPrec/year,AshDec Rhenania EUR/kgPrec,AshDec Rhenania EUR/year,PASCH kgPrec/year,PASCH EUR/kgPrec,PASCH EUR/year,LEACHPHOS kgPrec/year,LEACHPHOS EUR/kgPrec,LEACHPHOS EUR/year,RecoPhos kgPrec/year,RecoPhos EUR/kgPrec,RecoPhos EUR/year,Thermophos kgPrec/year,Thermophos EUR/kgPrec,Thermophos EUR/year
ID_WSER,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
375,Stelco Inc.,Lake Erie,Haldimand County,ON,42.7940,-80.1090,operational,3528.0,Haldimand-Norfolk,162.3130,164.7630,135.9750,125.5630,161.7000,202.1250,187.6450,thousand m3,517.432514,206.973005,37.714155,7.805812e+03,206.973005,36.435057,7.541073e+03,263.890582,57.764499,1.524351e+04,315.633833,56.068406,1.769709e+04,351.854109,170.905895,6.013394e+04,424.294661,12.5,5.303683e+03,444.991962,9.8,4.360921e+03,444.991962,9.9,4.405420e+03,408.771686,12.7,5.191400e+03,403.597361,13.1,5.287125e+03,450.166287,10.5,4.726746e+03,419.120336,10.7,4.484588e+03
503,Regional Municipality of Durham,Duffin Creek WPCP,Pickering,ON,43.8156,-79.0484,operational,3518.0,Durham,125615.3020,125187.3930,116781.0590,116277.1300,121734.3420,125252.9370,124847.1070,thousand m3,344266.846364,137706.738546,18.448872,2.540534e+06,137706.738546,11.307640,1.557138e+06,175576.091646,20.416175,3.584592e+06,210002.776282,17.400861,3.654229e+06,234101.455528,13.544746,3.170845e+06,282298.814019,12.5,3.528735e+06,296069.487873,9.8,2.901481e+06,296069.487873,9.9,2.931088e+06,271970.808628,12.7,3.454029e+06,268528.140164,13.1,3.517719e+06,299512.156337,10.5,3.144878e+06,278856.145555,10.7,2.983761e+06
1412,Bruce Power LP,Bruce Power,Tiverton,ON,44.3233,-81.5875,operational,3541.0,Bruce,376.6781,411.1270,411.6526,389.8714,391.9040,353.0745,340.7550,thousand m3,939.634502,375.853801,35.318545,1.327461e+04,375.853801,32.725064,1.229984e+04,479.213596,52.505425,2.516131e+04,573.177046,50.359252,2.886477e+04,638.951461,135.427231,8.653143e+04,770.500292,12.5,9.631254e+03,808.085672,9.8,7.919240e+03,808.085672,9.9,8.000048e+03,742.311257,12.7,9.427353e+03,732.914912,13.1,9.601185e+03,817.482017,10.5,8.583561e+03,761.103947,10.7,8.143812e+03
1436,The Corporation of the City of Barrie,Water Pollution Control Centre,Barrie,ON,44.3776,-79.6920,operational,3543.0,Simcoe,18676.6950,18465.9400,17527.8700,17707.7790,18509.8430,18110.8980,18197.0700,thousand m3,50178.558819,20071.423528,22.801913,4.576669e+05,20071.423528,15.992628,3.209948e+05,25591.064998,27.783995,7.110220e+05,30608.920880,24.610396,7.532977e+05,34121.419997,28.705023,9.794561e+05,41146.418232,12.5,5.143302e+05,43153.560584,9.8,4.229049e+05,43153.560584,9.9,4.272202e+05,39641.061467,12.7,5.034415e+05,39139.275879,13.1,5.127245e+05,43655.346172,10.5,4.583811e+05,40644.632643,10.7,4.348976e+05
7059,City of Sarnia,Water Pollution Control Centre,Sarnia,ON,42.9528,-82.4065,operational,3538.0,Lambton,8504.6068,7976.2739,8365.6310,9427.2610,10084.1690,10682.9760,11979.8060,thousand m3,33034.406089,13213.762436,23.874923,3.154776e+05,13213.762436,17.242457,2.278377e+05,16847.547105,29.705917,5.004718e+05,20150.987714,26.533706,5.346804e+05,22463.396141,33.788044,7.589942e+05,27088.212993,12.5,3.386027e+05,28409.589237,9.8,2.784140e+05,28409.589237,9.9,2.812549e+05,26097.180810,12.7,3.314342e+05,25766.836749,13.1,3.375456e+05,28739.933297,10.5,3.017693e+05,26757.868932,10.7,2.863092e+05
7391,DEPARTMENT OF NATIONAL DEFENCE,4th Can

### Slaughterhouse

In [33]:
SlaughterhouseCattle_Facilities = 7
SlaughterhouseSwine_Facilities = 6
SlaughterhousePoultry_Facilities = 17

SlaughterhouseCattle_PFlow = 2222E3 #kg per year
SlaughterhouseSwine_PFlow = 621E3 #kg per year
SlaughterhousePoultry_PFlow = 904E3 #kg per year

Slaughterhouse_Ontario_df = pd.DataFrame(data={'Facilities':[SlaughterhouseCattle_Facilities,SlaughterhouseSwine_Facilities,SlaughterhousePoultry_Facilities],
                                              'kgPin/year':[SlaughterhouseCattle_PFlow,SlaughterhouseSwine_PFlow,SlaughterhousePoultry_PFlow]},
                                         index=['Cattle','Swine','Poultry'])

Slaughterhouse_Ontario_df['Specific phosphorus flow (kg/year facility)'] = Slaughterhouse_Ontario_df['kgPin/year']/Slaughterhouse_Ontario_df['Facilities']/1000

Slaughterhouse_Ontario_df

,Facilities,kgPin/year,Specific phosphorus flow (kg/year facility)
Cattle,7,2222000.0,317.428571
Swine,6,621000.0,103.500000
Poultry,17,904000.0,53.176471


In [34]:
PRecovery_Slaughterhouse_Ontario_df = Slaughterhouse_Ontario_df.copy()

PFlow_Type = 'SlaughterhouseWasteLiquid'

SpecPRecCost_Param_SlaughterhouseWasteLiquid = SpecPRecCost_Param.filter(regex=PFlow_Type)


# for index, row in SpecPRecCost_Param_LiquidManure.iterrows():
for tech in SpecPRecCost_Param_SlaughterhouseWasteLiquid.columns:
    name = tech.rsplit('-')[0]
    PRecovery_Slaughterhouse_Ontario_df[name + ' kgPrec/year'] = (PRecovery_Slaughterhouse_Ontario_df['kgPin/year']*
                                                              SpecPRecCost_Param_SlaughterhouseWasteLiquid.loc['RecoveryEfficiency'][tech])
    
    TechParam = SpecPRecCost_Param.iloc[1:][tech].dropna().to_list()

    PRecovery_Slaughterhouse_Ontario_df[name + ' EUR/kgPrec'] = SpecPRecCost(PFlow_Type, name, PRecovery_Slaughterhouse_Ontario_df[name + ' kgPrec/year'],
                                                                         i=0.05, n=20, param=TechParam)
    
    PRecovery_Slaughterhouse_Ontario_df[name + ' EUR/year'] = PRecovery_Slaughterhouse_Ontario_df[name + ' EUR/kgPrec']*PRecovery_Slaughterhouse_Ontario_df[name + ' kgPrec/year']
    
    
    
PFlow_Type = 'SlaughterhouseWasteSolid'
SpecPRecCost_Param_SlaughterhouseWasteSolid = SpecPRecCost_Param.filter(regex=PFlow_Type)

for tech in SpecPRecCost_Param_SlaughterhouseWasteSolid.columns:
    name = tech.rsplit('-')[0]
    PRecovery_Slaughterhouse_Ontario_df[name + ' kgPrec/year'] = (PRecovery_Slaughterhouse_Ontario_df['kgPin/year']*
                                                              SpecPRecCost_Param_SlaughterhouseWasteSolid.loc['RecoveryEfficiency'][tech])
    
    TechParam = SpecPRecCost_Param.iloc[1:][tech].dropna().to_list()

    PRecovery_Slaughterhouse_Ontario_df[name + ' EUR/kgPrec'] = SpecPRecCost(PFlow_Type, name, PRecovery_Slaughterhouse_Ontario_df[name + ' kgPrec/year'],
                                                                         i=0.05, n=20, param=TechParam)
    
    PRecovery_Slaughterhouse_Ontario_df[name + ' EUR/year'] = PRecovery_Slaughterhouse_Ontario_df[name + ' EUR/kgPrec']*PRecovery_Slaughterhouse_Ontario_df[name + ' kgPrec/year']

    

PRecovery_Slaughterhouse_Ontario_df.to_csv('Results/PRecoveryAndCosts/PRecovery_Slaughterhouse_Ontario_df.csv')

PRecovery_Slaughterhouse_Ontario_df

,Facilities,kgPin/year,Specific phosphorus flow (kg/year facility),Multiform kgPrec/year,Multiform EUR/kgPrec,Multiform EUR/year,Ostara Pearl 500 kgPrec/year,Ostara Pearl 500 EUR/kgPrec,Ostara Pearl 500 EUR/year,Ostara Pearl 2K kgPrec/year,Ostara Pearl 2K EUR/kgPrec,Ostara Pearl 2K EUR/year,Ostara Pearl 10K kgPrec/year,Ostara Pearl 10K EUR/kgPrec,Ostara Pearl 10K EUR/year,EcoPhos kgPrec/year,EcoPhos EUR/kgPrec,EcoPhos EUR/year,AshDec depollution kgPrec/year,AshDec depollution EUR/kgPrec,AshDec depollution EUR/year,AshDec Rhenania kgPrec/year,AshDec Rhenania EUR/kgPrec,AshDec Rhenania EUR/year,PASCH kgPrec/year,PASCH EUR/kgPrec,PASCH EUR/year,LEACHPHOS kgPrec/year,LEACHPHOS EUR/kgPrec,LEACHPHOS EUR/year,RecoPhos kgPrec/year,RecoPhos EUR/kgPrec,RecoPhos EUR/year,Thermophos kgPrec/year,Thermophos EUR/kgPrec,Thermophos EUR/year
Cattle,7,2222000.0,317.428571,399960.0,31.868946,1.274630e+07,177760.0,32.211877,5.725983e+06,177760.0,22.596850,4.016816e+06,177760.0,20.114097,3.575482e+06,1577620.0,19.1,30132542.0,1644280.0,16.4,26966192.0,1644280.0,16.5,27130620.0,1510960.0,19.3,29161528.0,1488740.0,19.7,29328178.0,1666500.0,17.1,28497150.0,1555400.0,17.3,26908420.0
Swine,6,621000.0,103.500000,111780.0,32.075775,3.585430e+06,49680.0,34.174673,1.697798e+06,49680.0,25.614172,1.272512e+06,49680.0,31.751890,1.577434e+06,440910.0,19.1,8421381.0,459540.0,16.4,7536456.0,459540.0,16.5,7582410.0,422280.0,19.3,8150004.0,416070.0,19.7,8196579.0,465750.0,17.1,7964325.0,434700.0,17.3,7520310.0
Poultry,17,904000.0,53.176471,162720.0,32.364032,5.266275e+06,72320.0,33.463740,2.420098e+06,72320.0,22.479204,1.625696e+06,72320.0,26.695490,1.930618e+06,641840.0,19.1,12259144.0,668960.0,16.4,10970944.0,668960.0,16.5,11037840.0,614720.0,19.3,11864096.0,605680.0,19.7,11931896.0,678000.0,17.1,11593800.0,632800.0,17.3,10947440.0


### OFMSW (contains also food waste)

In [4]:
OFMSW_PFlow = 41E6 #kg per year

OFMSW_Ontario_df = pd.DataFrame(data={'kgPin/year':[OFMSW_PFlow]},
                                         index=['OFMSW'])

OFMSW_Ontario_df

,kgPin/year
OFMSW,41000000.0


In [36]:
PRecovery_OFMSW_Ontario_df = OFMSW_Ontario_df.copy()

PFlow_Type = 'OFMSW'

SpecPRecCost_Param_OFMSW = SpecPRecCost_Param.filter(regex=PFlow_Type)

# for index, row in SpecPRecCost_Param_LiquidManure.iterrows():
for tech in SpecPRecCost_Param_OFMSW.columns:
    name = tech.rsplit('-')[0]
    PRecovery_OFMSW_Ontario_df[name + ' kgPrec/year'] = (PRecovery_OFMSW_Ontario_df['kgPin/year']*
                                                              SpecPRecCost_Param_OFMSW.loc['RecoveryEfficiency'][tech])
    
    TechParam = SpecPRecCost_Param.iloc[1:][tech].dropna().to_list()

    PRecovery_OFMSW_Ontario_df[name + ' EUR/kgPrec'] = SpecPRecCost(PFlow_Type, name, PRecovery_OFMSW_Ontario_df[name + ' kgPrec/year'],
                                                                         i=0.05, n=20, param=TechParam)
    
    PRecovery_OFMSW_Ontario_df[name + ' EUR/year'] = PRecovery_OFMSW_Ontario_df[name + ' EUR/kgPrec']*PRecovery_OFMSW_Ontario_df[name + ' kgPrec/year']
    

PRecovery_OFMSW_Ontario_df.to_csv('Results/PRecoveryAndCosts/PRecovery_OFMSW_Ontario_df.csv')

PRecovery_OFMSW_Ontario_df

,kgPin/year,Extraction kgPrec/year,Extraction EUR/kgPrec,Extraction EUR/year,EcoPhos kgPrec/year,EcoPhos EUR/kgPrec,EcoPhos EUR/year,AshDec depollution kgPrec/year,AshDec depollution EUR/kgPrec,AshDec depollution EUR/year,AshDec Rhenania kgPrec/year,AshDec Rhenania EUR/kgPrec,AshDec Rhenania EUR/year,PASCH kgPrec/year,PASCH EUR/kgPrec,PASCH EUR/year,LEACHPHOS kgPrec/year,LEACHPHOS EUR/kgPrec,LEACHPHOS EUR/year,RecoPhos kgPrec/year,RecoPhos EUR/kgPrec,RecoPhos EUR/year,Thermophos kgPrec/year,Thermophos EUR/kgPrec,Thermophos EUR/year
OFMSW,41000000.0,38540000.0,24.8,955792000.0,33620000.0,10.93,367466600.0,35260000.0,8.23,290189800.0,35260000.0,8.33,293715800.0,32390000.0,11.13,360500700.0,31980000.0,11.53,368729400.0,35670000.0,8.93,318533100.0,33210000.0,9.13,303207300.0


# Result analysis

In [41]:
PRecovery_CattleCAFOs_Scenario1 = pd.read_csv('Results/PRecoveryAndCosts/PRecovery_CattleCAFOs_Scenario1.csv', sep=",", header=0, index_col=0)
PRecovery_CattleCAFOs_Scenario2 = pd.read_csv('Results/PRecoveryAndCosts/PRecovery_CattleCAFOs_Scenario2.csv', sep=",", header=0, index_col=0)
PRecovery_SwineCAFOs_Scenario1 = pd.read_csv('Results/PRecoveryAndCosts/PRecovery_SwineCAFOs_Scenario1.csv', sep=",", header=0, index_col=0)
PRecovery_WWWTPsLiquid_Ontario_df = pd.read_csv('Results/PRecoveryAndCosts/PRecovery_WWWTPsLiquid_Ontario_df.csv', sep=",", header=0, index_col=0)
PRecovery_WWWTPsSludge_Ontario_df = pd.read_csv('Results/PRecoveryAndCosts/PRecovery_WWWTPsSludge_Ontario_df.csv', sep=",", header=0, index_col=0)
PRecovery_Slaughterhouse_Ontario_df = pd.read_csv('Results/PRecoveryAndCosts/PRecovery_Slaughterhouse_Ontario_df.csv', sep=",", header=0, index_col=0)
PRecovery_OFMSW_Ontario_df = pd.read_csv('Results/PRecoveryAndCosts/PRecovery_OFMSW_Ontario_df.csv', sep=",", header=0, index_col=0)

PRecovery_Flows_list = ['PRecovery_CattleCAFOs_Scenario1','PRecovery_CattleCAFOs_Scenario2','PRecovery_SwineCAFOs_Scenario1',
                        'PRecovery_WWWTPsLiquid_Ontario_df','PRecovery_WWWTPsSludge_Ontario_df','PRecovery_Slaughterhouse_Ontario_df','PRecovery_OFMSW_Ontario_df']

PRecovery_OFMSW_Ontario_df
                                         

,kgPin/year,Extraction kgPrec/year,Extraction EUR/kgPrec,Extraction EUR/year,EcoPhos kgPrec/year,EcoPhos EUR/kgPrec,EcoPhos EUR/year,AshDec depollution kgPrec/year,AshDec depollution EUR/kgPrec,AshDec depollution EUR/year,AshDec Rhenania kgPrec/year,AshDec Rhenania EUR/kgPrec,AshDec Rhenania EUR/year,PASCH kgPrec/year,PASCH EUR/kgPrec,PASCH EUR/year,LEACHPHOS kgPrec/year,LEACHPHOS EUR/kgPrec,LEACHPHOS EUR/year,RecoPhos kgPrec/year,RecoPhos EUR/kgPrec,RecoPhos EUR/year,Thermophos kgPrec/year,Thermophos EUR/kgPrec,Thermophos EUR/year
OFMSW,41000000.0,38540000.0,24.8,955792000.0,33620000.0,10.93,367466600.0,35260000.0,8.23,290189800.0,35260000.0,8.33,293715800.0,32390000.0,11.13,360500700.0,31980000.0,11.53,368729400.0,35670000.0,8.93,318533100.0,33210000.0,9.13,303207300.0


In [49]:
a = PRecovery_OFMSW_Ontario_df.filter(regex='EUR/kgPrec').columns

SpecificPrecCost_df = PRecovery_OFMSW_Ontario_df[a]

MinCost = SpecificPrecCost_df.idxmin(axis=1)

MinCost_df = PRecovery_OFMSW_Ontario_df[MinCost]

MinCost_df

,AshDec depollution EUR/kgPrec
OFMSW,8.23
